In [1]:
import openai
import pandas as pd
import numpy as np
import json
import os
import re
import time
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate #where we change the AI "personality"
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv()
API_KEY = os.environ.get("OPENAI_API_KEY")

In [22]:
_ = load_dotenv(find_dotenv()) # read local .env file

#openai.api_key = os.getenv("api.txt")
COMPLETIONS_MODEL = "text-davinci-002"
BETTER_COMPLETIONS_MODEL = "text-davinci-003" #for my purposes, this is better
LONG_MODEL = "gpt-3.5-turbo-16k"
REGULAR_MODEL = "gpt-3.5-turbo"
GPT_4 = "gpt-4"

chat = ChatOpenAI(temperature=0.0,
                  openai_api_key = API_KEY,
                  verbose=True,
                  model_name=REGULAR_MODEL) #depending on how big of a task

#below, we give the AI a "personality"
template = """The following is a conversation between a human data scientist and an AI who specializes in data categorization. The AI is direct and provides concise responses. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Data Scientist: {input}
AI:"""

In [4]:
os.chdir('/Users/chrissoria/Documents/Research/openai_categorizations')
current_directory = os.getcwd()
print(current_directory)

/Users/chrissoria/Documents/Research/openai_categorizations


In [5]:
survey_participant_input = "a19i" #enter column name here

In [6]:
UCNets = pd.read_excel("/Users/chrissoria/Documents/Research/UCNets_Classification/data/Raw_Cond_for_Coding_all_waves.xlsx", engine='openpyxl',sheet_name="JOINT_DATA",usecols=[survey_participant_input])
UCNets = UCNets[survey_participant_input].dropna().unique()  # Drop NaN values and get unique elements

survey_participant_responses = '; '.join(str(item) for item in UCNets) #what we will feed to the model

UCNets = pd.DataFrame(UCNets, columns=[survey_participant_input])
UCNets[survey_participant_input] = UCNets[survey_participant_input].astype(str).str.lower()
UCNets[survey_participant_input] = UCNets[survey_participant_input].str.strip()
UCNets = UCNets[UCNets[survey_participant_input] != ''].reset_index(drop=True) #trimming all empty rows

UCNets = UCNets.iloc[:200]

UCNets.head()

,a19i
0,relocated back to east coast - closer to my sons
1,move in together with my partner
2,"out of living with my friends, and into living..."
3,to take a new job in new york city (both becau...
4,wanted to live in my own place outside my pare...


In [7]:
survey_input = UCNets['a19i'][10] #enter column name here
category = "to start living with or to stay with partner/spouse"

template_string = """A survey respondent was asked, "Why did you move?" \
They responded with: "{OBJECT}" \
First, please determine whether the reason for the move is {CATEGORY} \
Next, provide your answer as a 1 if yes and a 0 if no. Provide no further context \
If you're not sure, respond with a 0 and no further context."""

prompt_template = ChatPromptTemplate.from_template(template_string)
prompt_template.messages[0].prompt #this will show us our prompt template

GPT_Responses = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

In [8]:
print(GPT_Responses[0])

content='A survey respondent was asked, "Why did you move?" They responded with: "moved in with my fiance" First, please determine whether the reason for the move is to start living with or to stay with partner/spouse Next, provide your answer as a 1 if yes and a 0 if no. Provide no further context If you\'re not sure, respond with a 0 and no further context.' additional_kwargs={} example=False


In [9]:
for i in range(len(UCNets)):
    survey_input = UCNets['a19i'][i]  # enter column name here
    response = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

    response = chat(response)
    
    # Assuming a successful attempt means a non-empty response
    if response.content:
        print(f"Successful attempt for row number: {i}")
    
    UCNets.at[i, 'Partner_or_spouse'] = response.content

Successful attempt for row number: 0
Successful attempt for row number: 1
Successful attempt for row number: 2
Successful attempt for row number: 3
Successful attempt for row number: 4
Successful attempt for row number: 5
Successful attempt for row number: 6
Successful attempt for row number: 7
Successful attempt for row number: 8
Successful attempt for row number: 9
Successful attempt for row number: 10
Successful attempt for row number: 11
Successful attempt for row number: 12
Successful attempt for row number: 13
Successful attempt for row number: 14
Successful attempt for row number: 15
Successful attempt for row number: 16
Successful attempt for row number: 17
Successful attempt for row number: 18
Successful attempt for row number: 19
Successful attempt for row number: 20
Successful attempt for row number: 21
Successful attempt for row number: 22
Successful attempt for row number: 23
Successful attempt for row number: 24
Successful attempt for row number: 25
Successful attempt for

In [10]:
UCNets.to_csv('data/a19i_single_categorization.csv')

In [11]:
category = "relationship change (divorce, breakup, etc)"

for i in range(len(UCNets)):
    survey_input = UCNets['a19i'][i]  # enter column name here
    response = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

    response = chat(response)
    
    # Assuming a successful attempt means a non-empty response
    if response.content:
        print(f"Successful attempt for row number: {i}")
    
    UCNets.at[i, 'Relationship_end'] = response.content

Successful attempt for row number: 0
Successful attempt for row number: 1
Successful attempt for row number: 2
Successful attempt for row number: 3
Successful attempt for row number: 4
Successful attempt for row number: 5
Successful attempt for row number: 6
Successful attempt for row number: 7
Successful attempt for row number: 8
Successful attempt for row number: 9
Successful attempt for row number: 10
Successful attempt for row number: 11
Successful attempt for row number: 12
Successful attempt for row number: 13
Successful attempt for row number: 14
Successful attempt for row number: 15
Successful attempt for row number: 16
Successful attempt for row number: 17
Successful attempt for row number: 18
Successful attempt for row number: 19
Successful attempt for row number: 20
Successful attempt for row number: 21
Successful attempt for row number: 22
Successful attempt for row number: 23
Successful attempt for row number: 24
Successful attempt for row number: 25
Successful attempt for

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Successful attempt for row number: 169
Successful attempt for row number: 170
Successful attempt for row number: 171
Successful attempt for row number: 172
Successful attempt for row number: 173
Successful attempt for row number: 174
Successful attempt for row number: 175
Successful attempt for row number: 176
Successful attempt for row number: 177
Successful attempt for row number: 178
Successful attempt for row number: 179
Successful attempt for row number: 180
Successful attempt for row number: 181
Successful attempt for row number: 182
Successful attempt for row number: 183
Successful attempt for row number: 184
Successful attempt for row number: 185
Successful attempt for row number: 186
Successful attempt for row number: 187
Successful attempt for row number: 188
Successful attempt for row number: 189
Successful attempt for row number: 190
Successful attempt for row number: 191
Successful attempt for row number: 192
Successful attempt for row number: 193
Successful attempt for ro

In [12]:
UCNets.to_csv('data/a19i_single_categorization.csv')

In [14]:
category = "the respondent had a job/school/career change, including transferred"

for i in range(len(UCNets)):
    survey_input = UCNets['a19i'][i]  # enter column name here
    response = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

    response = chat(response)
    
    # Assuming a successful attempt means a non-empty response
    if response.content:
        print(f"Successful attempt for row number: {i}")
    
    UCNets.at[i, 'Own_job_school_career'] = response.content

Successful attempt for row number: 0
Successful attempt for row number: 1
Successful attempt for row number: 2
Successful attempt for row number: 3
Successful attempt for row number: 4
Successful attempt for row number: 5
Successful attempt for row number: 6
Successful attempt for row number: 7
Successful attempt for row number: 8
Successful attempt for row number: 9
Successful attempt for row number: 10
Successful attempt for row number: 11
Successful attempt for row number: 12
Successful attempt for row number: 13
Successful attempt for row number: 14
Successful attempt for row number: 15
Successful attempt for row number: 16
Successful attempt for row number: 17
Successful attempt for row number: 18
Successful attempt for row number: 19
Successful attempt for row number: 20
Successful attempt for row number: 21
Successful attempt for row number: 22
Successful attempt for row number: 23
Successful attempt for row number: 24
Successful attempt for row number: 25
Successful attempt for

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Successful attempt for row number: 140
Successful attempt for row number: 141
Successful attempt for row number: 142
Successful attempt for row number: 143
Successful attempt for row number: 144
Successful attempt for row number: 145
Successful attempt for row number: 146
Successful attempt for row number: 147
Successful attempt for row number: 148
Successful attempt for row number: 149
Successful attempt for row number: 150
Successful attempt for row number: 151
Successful attempt for row number: 152
Successful attempt for row number: 153
Successful attempt for row number: 154
Successful attempt for row number: 155
Successful attempt for row number: 156
Successful attempt for row number: 157
Successful attempt for row number: 158
Successful attempt for row number: 159
Successful attempt for row number: 160
Successful attempt for row number: 161
Successful attempt for row number: 162
Successful attempt for row number: 163
Successful attempt for row number: 164
Successful attempt for ro

In [15]:
UCNets.to_csv('data/a19i_single_categorization.csv')

In [16]:
category = "the respondent's partner's job/school/career change, including transferred"

for i in range(len(UCNets)):
    survey_input = UCNets['a19i'][i]  # enter column name here
    response = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

    response = chat(response)
    
    # Assuming a successful attempt means a non-empty response
    if response.content:
        print(f"Successful attempt for row number: {i}")
    
    UCNets.at[i, 'Partner_job_school_career'] = response.content

Successful attempt for row number: 0
Successful attempt for row number: 1
Successful attempt for row number: 2
Successful attempt for row number: 3
Successful attempt for row number: 4
Successful attempt for row number: 5
Successful attempt for row number: 6
Successful attempt for row number: 7
Successful attempt for row number: 8
Successful attempt for row number: 9
Successful attempt for row number: 10
Successful attempt for row number: 11
Successful attempt for row number: 12
Successful attempt for row number: 13
Successful attempt for row number: 14
Successful attempt for row number: 15
Successful attempt for row number: 16
Successful attempt for row number: 17
Successful attempt for row number: 18
Successful attempt for row number: 19
Successful attempt for row number: 20
Successful attempt for row number: 21
Successful attempt for row number: 22
Successful attempt for row number: 23
Successful attempt for row number: 24
Successful attempt for row number: 25
Successful attempt for

In [17]:
UCNets.to_csv('data/a19i_single_categorization.csv')

In [18]:
category = "financial reasons (rent is too expensive, pay raise, etc)"

for i in range(len(UCNets)):
    survey_input = UCNets['a19i'][i]  # enter column name here
    response = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

    response = chat(response)
    
    # Assuming a successful attempt means a non-empty response
    if response.content:
        print(f"Successful attempt for row number: {i}")
    
    UCNets.at[i, 'Financial'] = response.content

Successful attempt for row number: 0
Successful attempt for row number: 1
Successful attempt for row number: 2
Successful attempt for row number: 3
Successful attempt for row number: 4
Successful attempt for row number: 5
Successful attempt for row number: 6
Successful attempt for row number: 7
Successful attempt for row number: 8
Successful attempt for row number: 9
Successful attempt for row number: 10
Successful attempt for row number: 11
Successful attempt for row number: 12
Successful attempt for row number: 13
Successful attempt for row number: 14
Successful attempt for row number: 15
Successful attempt for row number: 16
Successful attempt for row number: 17
Successful attempt for row number: 18
Successful attempt for row number: 19
Successful attempt for row number: 20
Successful attempt for row number: 21
Successful attempt for row number: 22
Successful attempt for row number: 23
Successful attempt for row number: 24
Successful attempt for row number: 25
Successful attempt for

In [19]:
UCNets.to_csv('data/a19i_single_categorization.csv')

In [20]:
category = "Reasons related to specific housing features and preferences, different housing conditions"

for i in range(len(UCNets)):
    survey_input = UCNets['a19i'][i]  # enter column name here
    response = prompt_template.format_messages(
                    OBJECT=survey_input,
                    CATEGORY=category)

    response = chat(response)
    
    # Assuming a successful attempt means a non-empty response
    if response.content:
        print(f"Successful attempt for row number: {i}")
    
    UCNets.at[i, 'Housing_Features'] = response.content

Successful attempt for row number: 0
Successful attempt for row number: 1
Successful attempt for row number: 2
Successful attempt for row number: 3
Successful attempt for row number: 4
Successful attempt for row number: 5
Successful attempt for row number: 6
Successful attempt for row number: 7
Successful attempt for row number: 8
Successful attempt for row number: 9
Successful attempt for row number: 10
Successful attempt for row number: 11
Successful attempt for row number: 12
Successful attempt for row number: 13
Successful attempt for row number: 14
Successful attempt for row number: 15
Successful attempt for row number: 16
Successful attempt for row number: 17
Successful attempt for row number: 18
Successful attempt for row number: 19
Successful attempt for row number: 20
Successful attempt for row number: 21
Successful attempt for row number: 22
Successful attempt for row number: 23
Successful attempt for row number: 24
Successful attempt for row number: 25
Successful attempt for

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Successful attempt for row number: 48
Successful attempt for row number: 49
Successful attempt for row number: 50
Successful attempt for row number: 51


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised ServiceUnavailableError: The server is overloaded or not ready yet..


Successful attempt for row number: 52
Successful attempt for row number: 53
Successful attempt for row number: 54
Successful attempt for row number: 55
Successful attempt for row number: 56
Successful attempt for row number: 57
Successful attempt for row number: 58
Successful attempt for row number: 59
Successful attempt for row number: 60
Successful attempt for row number: 61
Successful attempt for row number: 62
Successful attempt for row number: 63
Successful attempt for row number: 64
Successful attempt for row number: 65
Successful attempt for row number: 66
Successful attempt for row number: 67
Successful attempt for row number: 68
Successful attempt for row number: 69
Successful attempt for row number: 70
Successful attempt for row number: 71
Successful attempt for row number: 72
Successful attempt for row number: 73
Successful attempt for row number: 74
Successful attempt for row number: 75
Successful attempt for row number: 76
Successful attempt for row number: 77
Successful a

...

Successful attempt for row number: 121
Successful attempt for row number: 122
Successful attempt for row number: 123
Successful attempt for row number: 124
Successful attempt for row number: 125
Successful attempt for row number: 126
Successful attempt for row number: 127
Successful attempt for row number: 128
Successful attempt for row number: 129
Successful attempt for row number: 130
Successful attempt for row number: 131
Successful attempt for row number: 132
Successful attempt for row number: 133
Successful attempt for row number: 134
Successful attempt for row number: 135
Successful attempt for row number: 136
Successful attempt for row number: 137
Successful attempt for row number: 138
Successful attempt for row number: 139


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Successful attempt for row number: 140
Successful attempt for row number: 141
Successful attempt for row number: 142
Successful attempt for row number: 143
Successful attempt for row number: 144
Successful attempt for row number: 145
Successful attempt for row number: 146
Successful attempt for row number: 147
Successful attempt for row number: 148
Successful attempt for row number: 149
Successful attempt for row number: 150
Successful attempt for row number: 151
Successful attempt for row number: 152
Successful attempt for row number: 153
Successful attempt for row number: 154
Successful attempt for row number: 155


Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIError: HTTP code 502 from API (<html>
<head><title>502 Bad Gateway</title></head>
<body>
<center><h1>502 Bad Gateway</h1></center>
<hr><center>cloudflare</center>
</body>
</html>
).


Successful attempt for row number: 156
Successful attempt for row number: 157
Successful attempt for row number: 158
Successful attempt for row number: 159
Successful attempt for row number: 160
Successful attempt for row number: 161
Successful attempt for row number: 162
Successful attempt for row number: 163
Successful attempt for row number: 164
Successful attempt for row number: 165
Successful attempt for row number: 166
Successful attempt for row number: 167
Successful attempt for row number: 168
Successful attempt for row number: 169
Successful attempt for row number: 170
Successful attempt for row number: 171
Successful attempt for row number: 172
Successful attempt for row number: 173
Successful attempt for row number: 174
Successful attempt for row number: 175
Successful attempt for row number: 176
Successful attempt for row number: 177
Successful attempt for row number: 178
Successful attempt for row number: 179
Successful attempt for row number: 180
Successful attempt for ro

In [21]:
UCNets.to_csv('data/a19i_single_categorization.csv',index = False)